In [ ]:
import torch
# GPUが使用可能か判断
if torch.cuda.is_available():
    print('gpu is available')
else:
    raise Exception('gpu is NOT available')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# !pip install transformers[torch] datasets
# !pip install fugashi
# !pip install ipadic
# !pip install sentencepiece

In [ ]:
# !pip install git+https://github.com/huggingface/accelerate
# !pip install --upgrade transformers

In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import pandas as pd
import torch
import random

In [ ]:
from transformers.trainer_utils import set_seed

# 乱数シードを42に固定
set_seed(42)

In [ ]:
from pprint import pprint
from datasets import load_dataset

# Hugging Face Hub上ののリポジトリからデータを読み込む
train_dataset = load_dataset("tyqiangz/multilingual-sentiments", "japanese", split="train")
valid_dataset = load_dataset("tyqiangz/multilingual-sentiments", "japanese", split="validation")
# pprintで見やすく表示する
pprint(train_dataset)
pprint(valid_dataset)

In [ ]:
# # 実験のためデータセットを縮小したい場合はコチラを有効化
# train_dataset = train_dataset.select(
#     random.sample(range(train_dataset.num_rows), k=500))
# valid_dataset = valid_dataset.select(
#     random.sample(range(valid_dataset.num_rows), k=500))
# pprint(train_dataset)
# pprint(valid_dataset)

In [ ]:
train_dataset[0]

In [ ]:
# トークナイザのロード
model_name = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# 大規模言語モデル入門に記載されているコードで視覚化して検証
from collections import Counter
import japanize_matplotlib
import matplotlib.pyplot as plt
import numpy as np
from datasets import Dataset
from tqdm import tqdm

# plt.rcParams["font.size"] = 18  # 文字サイズを大きくする

def visualize_text_length(dataset: Dataset):
    """データセット中のテキストのトークン数の分布をグラフとして描画"""
    # データセット中のテキストの長さを数える
    length_counter = Counter()
    for data in tqdm(dataset):
        length = len(tokenizer.tokenize(data["text"]))
        length_counter[length] += 1
    # length_counterの値から棒グラフを描画する
    plt.bar(length_counter.keys(), length_counter.values(), width=1.0)
    plt.xlabel("トークン数")
    plt.ylabel("事例数")
    plt.show()
    print('max:',max(length_counter.keys()))
    PERCENT=95
    print(f'{PERCENT}%:',np.percentile([k for k in length_counter.keys()], PERCENT))
    PERCENT=80
    print(f'{PERCENT}%:',np.percentile([k for k in length_counter.keys()], PERCENT))

visualize_text_length(train_dataset)

In [ ]:
# トークナイズ関数
# text列をトークナイズ
# label列をlabels列にセット
def preprocess_text(batch):
    encoded_batch = tokenizer(batch['text'], max_length=512)
    encoded_batch['labels'] = batch['label']
    return encoded_batch

In [ ]:
# トークナイズ処理（既存の列は削除）
encoded_train_dataset = train_dataset.map(
    preprocess_text,
    remove_columns=train_dataset.column_names,
)
encoded_valid_dataset = valid_dataset.map(
    preprocess_text,
    remove_columns=valid_dataset.column_names,
)

In [ ]:
# お試し
from pprint import pprint
pprint(encoded_train_dataset[0], width=1000)

In [ ]:
# ミニバッチ構築
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# お試し
batch_inputs = data_collator(encoded_train_dataset[0:4])
pprint({name: tensor.size() for name, tensor in batch_inputs.items()})

In [ ]:
# モデルの準備
from transformers import AutoModelForSequenceClassification

class_label = train_dataset.features["label"]
label2id = {label: id for id, label in enumerate(class_label.names)}
id2label = {id: label for id, label in enumerate(class_label.names)}
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=class_label.num_classes,
    label2id=label2id,  # ラベル名からIDへの対応を指定
    id2label=id2label,  # IDからラベル名への対応を指定
)
print(type(model).__name__)

In [ ]:
# お試し
pprint(model.forward(**data_collator(encoded_train_dataset[0:4])))

In [ ]:
# 訓練の実行
from transformers import TrainingArguments

BATCH_SIZE = 32
training_args = TrainingArguments(
    output_dir="output_multilingual",  # 結果の保存フォルダ
    per_device_train_batch_size=BATCH_SIZE,  # 訓練時のバッチサイズ
    per_device_eval_batch_size=BATCH_SIZE,  # 評価時のバッチサイズ
    gradient_accumulation_steps=2,  # 勾配累積
    learning_rate=2e-5,  # 学習率
    lr_scheduler_type="constant",  # 学習率スケジューラの種類
    warmup_ratio=0.1,  # 学習率のウォームアップの長さを指定
    num_train_epochs=3,  # エポック数
    save_strategy="epoch",  # チェックポイントの保存タイミング
    logging_strategy="epoch",  # ロギングのタイミング
    evaluation_strategy="epoch",  # 検証セットによる評価のタイミング
    load_best_model_at_end=True,  # 訓練後に開発セットで最良のモデルをロード
    metric_for_best_model="accuracy",  # 最良のモデルを決定する評価指標
    fp16=True,  # 自動混合精度演算の有効化
)

In [ ]:
# メトリクスの定義
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_valid_dataset,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
)
trainer.train()

In [ ]:
# epoch: 100, early stopping
# 訓練の実行
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="output_multilingual",  # 結果の保存フォルダ
    per_device_train_batch_size=32,  # 訓練時のバッチサイズ
    per_device_eval_batch_size=32,  # 評価時のバッチサイズ
    learning_rate=2e-5,  # 学習率
    lr_scheduler_type="constant",  # 学習率スケジューラの種類
    warmup_ratio=0.1,  # 学習率のウォームアップの長さを指定
    num_train_epochs=100,  # エポック数
    save_strategy="epoch",  # チェックポイントの保存タイミング
    logging_strategy="epoch",  # ロギングのタイミング
    evaluation_strategy="epoch",  # 検証セットによる評価のタイミング
    load_best_model_at_end=True,  # 訓練後に開発セットで最良のモデルをロード
    metric_for_best_model="eval_loss",  # 最良のモデルを決定する評価指標
    greater_is_better=False,            # eval_lossは小さいほどよい
    fp16=True,  # 自動混合精度演算の有効化
)

In [ ]:
from transformers import Trainer
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_valid_dataset,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)
trainer.train()

In [ ]:
history_df = pd.DataFrame(trainer.state.log_history)
history_df.to_csv('base_line/mullingual_baseline_history.csv')

In [ ]:
from sklearn.linear_model import LinearRegression

def linear_regression(history_df):
    y = history_df['eval_loss'].dropna().values
    x = np.arange(len(y)).reshape(-1, 1)
    linear = LinearRegression().fit(x, y)
    return linear

In [ ]:
import matplotlib.pyplot as plt

def show_graph(df, suptitle, regression, output='output.png'):
    suptitle_size = 23
    graph_title_size = 20
    legend_size = 18
    ticks_size = 13
    # 学習曲線
    fig = plt.figure(figsize=(20, 5))
    # plt.suptitle(','.join([f'{e}: {parameters[e]}' for e in parameters.keys()]), fontsize=suptitle_size)
    plt.suptitle(suptitle, fontsize=suptitle_size)
    # Train Loss
    plt.subplot(131)
    plt.title('Train Loss', fontsize=graph_title_size)
    plt.plot(df['loss'].dropna(), label='train')
    plt.legend(fontsize=legend_size)
    plt.yticks(fontsize=ticks_size)
    # Validation Loss
    plt.subplot(132)
    reg_str = f'$y={round(regression.coef_[0],5)}*x+{round(regression.intercept_,3)}$'
    plt.title(f'Val Loss', fontsize=graph_title_size)
    y = df['eval_loss'].dropna().values
    x = np.arange(len(y)).reshape(-1, 1)
    pred = regression.coef_ * x.ravel() + regression.intercept_  # 線形回帰直線
    plt.plot(y, color='tab:orange', label='val')
    plt.plot(pred, color='green', label='pred')
    plt.legend(fontsize=legend_size)
    plt.xlabel(reg_str, fontsize=ticks_size)
    plt.yticks(fontsize=ticks_size)
    # Accuracy/F1
    plt.subplot(133)
    plt.title('eval Accuracy/F1', fontsize=graph_title_size)
    plt.plot(df['eval_accuracy'].dropna(), label='accuracy')
    plt.plot(df['eval_f1'].dropna(), label='F1')
    plt.legend(fontsize=legend_size)
    plt.yticks(fontsize=ticks_size)
    plt.tight_layout()
    # plt.show()
    plt.savefig(output)

In [ ]:
# 結果を表示
suptitle = 'batch:32, lr:2e-5, type:constant'
reg = linear_regression(history_df)
show_graph(history_df, suptitle, reg, 'base_line/mullingual_output.png')